## Withdrawal precompile reentrancy tests

### Scenario A: `TokenProxyTester` withdraws tickets twice during withdrawal, but it has these tickets
- setup:
    - tester originated with callsCount=0, amount=10
    - deposit 1000 tickets to the proxy from L1 via kernel
    - tester resetup to have callsCount=1, amount=10
- test:
    - user calls `fa_withdrawal_precompile` to withdraw 1 ticket from the tester
        * tester withdraws 10 more tickets in the internal call
    - check transaction succeed
    - check that 989 tickets left on tester
    - check that 2 withdrawal events created, each for 2 tickets
    - check that 2 outbox messages created
    - deposit 21 tickets to the proxy from L1 via kernel
        * tester withdraws 10 more tickets in the internal call
    - check that 1 outbox message created
    - check that 1 deposit event created
    - check that 1000 tickets left on tester
    - user tries to withdraw > 990 tickets from tester and transaction fails
    - user tries tp withdraw 990 tickets and succeed
        * tester withdraws 10 more tickets in the internal call
    - check that two withdrawal events and 2 outbox messages created
    - deposit 15 tickets to the proxy from L1 via kernel
        * tester tries to withdraw 10 more tickets in the internal call but not succeed
    - check deposit succeed, deposit event created and tester recieves 15 tickets

In [1]:
from docs.scenarios.setup import *

accounts = setup()
web3, etherlink_account, tezos_account = accounts
_, ticketer, _, _, ticket_router_tester = load_contracts(*accounts)

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `209.947907 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `503.080857969 ꜩ`


### Deploy TokenProxyTester:
- This is the contract that can be used in the replacement of the `ERC20Proxy` makes reentrant calls to the `fa_withdrawal_precompile` during both `deposit` and `withdrawal` calls.

In [2]:
calls_count = 1
withdraw_amount = 10
routing_info = make_withdrawal_routing_info(tezos_account, ticketer)
ticketer_address_bytes = tezos_address_to_bytes(ticketer.address)
# TODO: consider adding `to_bytes` method
ticket_content_bytes = bytes.fromhex(ticketer.read_content().to_bytes_hex())

token_proxy_tester = TokenProxyTesterHelper.originate_from_file(
    web3=web3,
    filename=make_filename('TokenProxyTester'),
    account=etherlink_account,
    constructor_args=(
        # TODO: the kernel address added as an initial Ticket Owner
        # it should be replaced with the contract that will be used as an ERC20Proxy:
        KERNEL_ADDRESS,
        FA_WITHDRAWAL_PRECOMPILE,
        routing_info,
        withdraw_amount,
        ticketer_address_bytes,
        ticket_content_bytes,
        calls_count,
    )
)

In [3]:
token_proxy_tester.address

'0x4A338A3Cb0017A3E0E9156397d98B97795032C72'

### Work in progress ...
---

- Reload TokenProxyTester

In [33]:
token_proxy_tester = TokenProxyTesterHelper.from_address(
    web3,
    etherlink_account,
    '0x4A338A3Cb0017A3E0E9156397d98B97795032C72'
)
token_proxy_tester.address

'0x4A338A3Cb0017A3E0E9156397d98B97795032C72'

In [34]:
opg_hash = setup_ticket_router_tester_to_rollup_deposit(
    ticket_router_tester=ticket_router_tester,
    target=SMART_ROLLUP_ADDRESS,
    receiver=token_proxy_tester,
    # TODO: make no router (optional)
    router=token_proxy_tester,
)

Setting up TicketRouterTester to the rollupDeposit mode:
  - Target: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
  - Routing Info: `0x4a338a3cb0017a3e0e9156397d98b97795032c724a338a3cb0017a3e0e9156397d98b97795032c72`
Successfully set, tx hash: `oovwcRdcyqsoihB9gjPcksC16CsphrWjoerGdvrfvG49kNro3vB`


In [35]:
ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(1_000)

opg_hash = transfer_ticket(
    ticket=ticket,
    receiver=ticket_router_tester,
)

Transfering ticket:
  - Owner: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw`
  - Ticket content: `0x0707000005090a000000cc0502000000c607040100000010636f6e74726163745f616464726573730a000000244b54315632616b314d664e643377346f794b44363465685955374b34437270556344475207040100000008646563696d616c730a0000000136070401000000046e616d650a0000001454657374205465746865722055534420763134310704010000000673796d626f6c0a0000000d544553545f555344745f31343107040100000008746f6b656e5f69640a00000001300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `1_000`
  - Receiver: `KT1KKHDuPeZ4KptN591TZ9UiCeKhHpKqaE3Y`
Successfully transfered, tx hash: `onjrQmSV32UfcRzgmushcdJnggEd2eMamZDR5SzWmnPtHWwKmcL`


### Getting ticket table value:
- Checking that token_proxy_tester received 1000 tickets:

In [43]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

1000

### Setup to make 3 calls:
- TODO: need to find a way to simplify this setup, maybe reuse some logic from deployment, as this reuse the same parameters:

In [44]:
withdraw_amount = 1
calls_count = 3

# NOTE: this was the values in previous test:
# withdraw_amount = 10
# calls_count = 1
receiver = tezos_account
ticket = ticketer.read_ticket()
ticketer_address_bytes = bytes.fromhex(make_address_bytes(ticket.ticketer))
ticket_content_bytes = bytes.fromhex(ticket.content.to_bytes_hex())

result = token_proxy_tester.set_parameters(
    # TODO: try another scenario where ticket owner is another token_proxy_tester (ping-pong withdrawals)
    ticket_owner=token_proxy_tester.address,
    withdrawal_precompile=FA_WITHDRAWAL_PRECOMPILE,
    routing_info=make_withdrawal_routing_info(receiver, ticketer),
    amount=withdraw_amount,
    ticketer=ticketer_address_bytes,
    content=ticket_content_bytes,
    calls_count=calls_count
)

result

AttributeDict({'transactionHash': HexBytes('0x6cd775e722a61f689f0dd8bca36f8a3d8d144b2d1dbfc31fc099ddb535dc7e64'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x78536b2401ed604f53c165993552edf76d2527af6ef3d912cd86015d1f249b8b'),
 'blockNumber': 6271347,
 'from': '0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677',
 'to': '0x4A338A3Cb0017A3E0E9156397d98B97795032C72',
 'cumulativeGasUsed': 2977467,
 'effectiveGasPrice': 1000000000,
 'gasUsed': 2977467,
 'logs': [],
 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [45]:
token_proxy_tester.address

'0x4A338A3Cb0017A3E0E9156397d98B97795032C72'

In [46]:
withdraw.callback(
    erc20_proxy_address=token_proxy_tester.address,
    tezos_side_router_address=ticketer.address,
    amount=1,
    # TODO: consider changing this to ticketer_address_bytes_hex ?
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    # TODO: consider changing this to ticket_content_bytes_hex ?
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=FA_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making FA withdrawal, ERC20 token: `0x4A338A3Cb0017A3E0E9156397d98B97795032C72`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Ticket owner: `0x4A338A3Cb0017A3E0E9156397d98B97795032C72`
      * Receiver: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Router: `KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw`
      * Routing info: `0x0000d1b03118754a8e193b3e5d3a7cded976f4fd425601c0137ec3632f70c119ea958dca84cdaafa42024000`
      * Amount: `1`
      * Ticketer address bytes: `0x01c0137ec3632f70c119ea958dca84cdaafa42024000`
      * Content bytes: `0x0707000005090a000000cc0502000000c607040100000010636f6e74726163745f616464726573730a000000244b54315632616b314d664e643377346f794b44363465685955374b34437270556344475207040100000008646563696d616c730a0000000136070401000000046e616d650a0000001454657374205465746865722055534420763134310704010000000673796d626f6c0a0000000d544553545f555344745f3134310704010000000874

'0xb00b86e3a978e2273530e89833d7aad00e453af5fdffae84be601c3b548aa6b9'

- NOTE: need to wait until rollup processes transaction, not sure what is the best way to do this, probably simple time.sleep would work?

- checking that it has 4 tokens less (1_000 - 3 - 1 = 996)

In [53]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

996

In [54]:
ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(7)

opg = transfer_ticket(
    ticket=ticket,
    receiver=ticket_router_tester,
)

Transfering ticket:
  - Owner: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw`
  - Ticket content: `0x0707000005090a000000cc0502000000c607040100000010636f6e74726163745f616464726573730a000000244b54315632616b314d664e643377346f794b44363465685955374b34437270556344475207040100000008646563696d616c730a0000000136070401000000046e616d650a0000001454657374205465746865722055534420763134310704010000000673796d626f6c0a0000000d544553545f555344745f31343107040100000008746f6b656e5f69640a00000001300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `7`
  - Receiver: `KT1KKHDuPeZ4KptN591TZ9UiCeKhHpKqaE3Y`
Successfully transfered, tx hash: `opCQvg7uaVqQhU5ALhqVpZLHA31eXwY1Dw7spHF2f5WokZwGq8X`


- TODO: check that amount is expected, 1000

In [55]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

1000

### The following checked with 1000 > amount > 997 and it fails, but 997 works:

In [58]:
result = withdraw.callback(
    erc20_proxy_address=token_proxy_tester.address,
    tezos_side_router_address=ticketer.address,
    amount=997,
    # TODO: consider changing this to ticketer_address_bytes_hex ?
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    # TODO: consider changing this to ticket_content_bytes_hex ?
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=FA_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making FA withdrawal, ERC20 token: `0x4A338A3Cb0017A3E0E9156397d98B97795032C72`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Ticket owner: `0x4A338A3Cb0017A3E0E9156397d98B97795032C72`
      * Receiver: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Router: `KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw`
      * Routing info: `0x0000d1b03118754a8e193b3e5d3a7cded976f4fd425601c0137ec3632f70c119ea958dca84cdaafa42024000`
      * Amount: `997`
      * Ticketer address bytes: `0x01c0137ec3632f70c119ea958dca84cdaafa42024000`
      * Content bytes: `0x0707000005090a000000cc0502000000c607040100000010636f6e74726163745f616464726573730a000000244b54315632616b314d664e643377346f794b44363465685955374b34437270556344475207040100000008646563696d616c730a0000000136070401000000046e616d650a0000001454657374205465746865722055534420763134310704010000000673796d626f6c0a0000000d544553545f555344745f31343107040100000008

- TODO: check expected ticket count is 0

In [59]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

0

- TODO: rewrite this:
- NEXT: deposit when contract tries to make withdrawal when there are no tickets, but deposit still should succeed because even if it failed the receiver should get tickets and receiver is the TokenProxyTester:

In [60]:
ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(15)

opg = transfer_ticket(
    ticket=ticket,
    receiver=ticket_router_tester,
)

Transfering ticket:
  - Owner: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw`
  - Ticket content: `0x0707000005090a000000cc0502000000c607040100000010636f6e74726163745f616464726573730a000000244b54315632616b314d664e643377346f794b44363465685955374b34437270556344475207040100000008646563696d616c730a0000000136070401000000046e616d650a0000001454657374205465746865722055534420763134310704010000000673796d626f6c0a0000000d544553545f555344745f31343107040100000008746f6b656e5f69640a00000001300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `15`
  - Receiver: `KT1KKHDuPeZ4KptN591TZ9UiCeKhHpKqaE3Y`
Successfully transfered, tx hash: `oo8uNFsNaWynanXFrV3yqYPrrGauVkVHvbMqFsvg1av3JJ5CoyF`


- TODO: check expected tickets is 15

In [72]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

15

## ?
- Scenario B: `TokenProxyTester` withdraws tickets twice during withdrawal, but it has not enough tickers
- setup:
    - tester originated with callsCount=0
    - deposit 10 tickets to the proxy from L1 via kernel
    - tester resetup to have callsCount=1, amount=10
- test:
    - user calls `fa_withdrawal_precompile` to withdraw 1 ticket from proxy
        * tester tries to withdraw 10 tickets more in the internal call
    - check that transaction failed
    - check that `fa_withdrawal_precompile` not broken after this failure

## ?
- Scenario C: `TokenProxyTester` withdraws tickets during deposit
- setup:
    - tester originated with callsCount=1, amount=11
- test:
    - deposit 10 tickets to the proxy from L1 via kernel
        * tester tries to withdraw 11 tickets and fails

## ?
- Scenario D: with successful withdrawal during deposit (withdraw less than deposit)
- TODO: consider making abstract test for deposit/withdrawal with different setups?
    - some kind of fuzz testing? parametrize all of the scenarios into one and check them all? and try to find more?